In [1]:
cd C:/Users/frankh/python/regression-challenge

C:\Users\frankh\python\regression-challenge


In [2]:
pwd

'C:\\Users\\frankh\\python\\regression-challenge'

In [3]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import tensorflow        as tf
import os
import random

from tensorflow	                import keras
from tensorflow.keras.models    import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras           import layers
from tensorflow.keras.layers    import InputLayer

from sklearn.model_selection     import train_test_split
from sklearn.preprocessing       import StandardScaler
from sklearn.preprocessing       import Normalizer
from sklearn.metrics             import r2_score
from sklearn.compose             import ColumnTransformer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks  import EarlyStopping

debug = False

def reset_seeds():
    seed_value = 1234
    random.seed(seed_value)
    np.random.seed(seed_value) 
    tf.random.set_seed(seed_value)
    tf.compat.v1.set_random_seed(seed_value)
    tf.random.set_seed(seed_value)

In [4]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())
reset_seeds()

[]
False


In [5]:
dataset = pd.read_csv("admissions_data.csv")
print(dataset.describe(include='all'))
print(dataset.head())
print(dataset.columns)

       Serial No.   GRE Score  TOEFL Score  University Rating         SOP  \
count  500.000000  500.000000   500.000000         500.000000  500.000000   
mean   250.500000  316.472000   107.192000           3.114000    3.374000   
std    144.481833   11.295148     6.081868           1.143512    0.991004   
min      1.000000  290.000000    92.000000           1.000000    1.000000   
25%    125.750000  308.000000   103.000000           2.000000    2.500000   
50%    250.500000  317.000000   107.000000           3.000000    3.500000   
75%    375.250000  325.000000   112.000000           4.000000    4.000000   
max    500.000000  340.000000   120.000000           5.000000    5.000000   

            LOR         CGPA    Research  Chance of Admit   
count  500.00000  500.000000  500.000000         500.00000  
mean     3.48400    8.576440    0.560000           0.72174  
std      0.92545    0.604813    0.496884           0.14114  
min      1.00000    6.800000    0.000000           0.34000  
2

In [6]:
features = dataset.iloc[:,1:-1]
labels   = dataset.iloc[:,-1]
print(features.columns)

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research'],
      dtype='object')


In [7]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [8]:
def print_column_values(dataset):
    if debug :
        print(dataset['GRE Score'].unique())
        print(dataset['TOEFL Score'].unique())
        print(dataset['University Rating'].unique())
        print(dataset['SOP'].unique())
        print(dataset['LOR '].unique())
        print(dataset['CGPA'].unique())
        print(dataset['Research'].unique())

In [9]:
debug = False
print_column_values(features)

In [10]:
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
print(numerical_columns)

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research'],
      dtype='object')


In [11]:
ct = ColumnTransformer([('scale',StandardScaler(),  numerical_columns         )], remainder='passthrough')

In [12]:
debug = False
print_column_values(features)

In [13]:
features_train_scale = ct.fit_transform(features_train)
features_test_scale  = ct.transform(features_test)
features_train_scale = pd.DataFrame(features_train_scale, columns = features_train.columns)
features_test_scale  = pd.DataFrame(features_test_scale, columns = features_test.columns)

In [14]:
debug = False
print_column_values(features_train_scale)

In [15]:
val_mse_list = []
val_mae_list = []
lvl_1_list   = [2**I for I in range(2,7)]
lvl_2_list   = [2**J for J in range(1,6)]
epochs_list  = [10*E for E in range(1,11)]
print(lvl_1_list)
print(lvl_2_list)
print(epochs_list)
# MIN: 16 2 20 0.0032181069254875183
# MIN: 4 32 90 0.03884091228246689
# MIN: 4 32 90 0.03884091228246689
# MIN: 4 32 90 0.03884091228246689
lvl_1_list  = [4]
lvl_2_list  = [32]
epochs_list = [90]
for I in lvl_1_list:
    for J in lvl_2_list: 
        for E in epochs_list:
            reset_seeds()
            model = Sequential()
            num_features = features_train_scale.shape[1]
            inputs = InputLayer(input_shape=(num_features,))
            model.add(inputs)
            model.add(layers.Dense(I, activation='relu'))
            model.add(layers.Dense(J, activation='relu'))        
            model.add(layers.Dense(1))
            opt = Adam(learning_rate=0.01)
            model.compile(loss='mse', metrics=['mae'], optimizer=opt)
            stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
            model.fit(features_train_scale, labels_train, epochs=E, batch_size=3, verbose=0, validation_split=0.2, callbacks=[stop])
            val_mse, val_mae = model.evaluate(features_test_scale, labels_test, verbose=1)
            print(model.layers)
            print(model.summary())
            print(val_mse, val_mae)
            val_mse_list.append([I, J, E, val_mse])
            val_mae_list.append([I, J, E, val_mae])


[4, 8, 16, 32, 64]
[2, 4, 8, 16, 32]
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
4/4 [==============================] - 0s 2ms/step - loss: 0.0033 - mae: 0.0388
[<keras.layers.core.dense.Dense object at 0x000001BCD8152108>, <keras.layers.core.dense.Dense object at 0x000001BCD81AAFC8>, <keras.layers.core.dense.Dense object at 0x000001BCD81CB848>]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 32        
                                                                 
 dense_1 (Dense)             (None, 32)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
__________________________________________

In [16]:
minI = 0
minJ = 0
minE = 0
minMAE = 1
for I,J,E,MAE in val_mae_list:
    # print(str(I), str(J), str(E), str(MSE))
    if MAE<minMAE:
        minI = I
        minJ = J
        minE = E
        minMAE = MAE
print('MIN:', str(minI), str(minJ), str(minE), str(minMAE))

MIN: 4 32 90 0.03884091228246689


In [54]:
features_predict = pd.DataFrame(model.predict(features_test_scale))
print(type(features_predict))
print(features_predict.iloc[:,0])

4/4 [==============================] - 0s 2ms/step
<class 'pandas.core.frame.DataFrame'>
0     0.661321
1     0.703204
2     0.954549
3     0.713874
4     0.817477
        ...   
95    0.833468
96    0.839972
97    0.507549
98    0.727970
99    0.588899
Name: 0, Length: 100, dtype: float32


In [55]:
labels_test = pd.DataFrame(labels_test)
print(type(labels_test))
print(labels_test.iloc[:,0])

<class 'pandas.core.frame.DataFrame'>
304    0.62
340    0.75
47     0.89
67     0.57
479    0.79
       ... 
11     0.84
192    0.86
92     0.34
221    0.75
110    0.61
Name: Chance of Admit , Length: 100, dtype: float64


In [61]:
# pd.merge(features_predict.rename(columns={'0':'COL'}), labels_test.rename(columns={'Chance of Admit ':'COL'}))